What state / chamber ensembles have anti majoritarian plans and how many?

In [7]:
import os
import pandas as pd
from collections import defaultdict

from rdapy import DISTRICTS_BY_STATE

from rdametrics import *

Load the all-up scores dataframe from disk

In [2]:
scores_path: str = "~/local/beta-ensembles/dataframe/contents/scores_df.parquet"
scores_df = pd.read_parquet(os.path.expanduser(scores_path))
scores_df.head()

,map,estimated_vote_pct,pr_deviation,estimated_seats,fptp_seats,disproportionality,efficiency_gap_wasted_votes,efficiency_gap_statewide,efficiency_gap,seats_bias,...,proportional_coalitions,minority,county_splitting,district_splitting,counties_split,county_splits,splitting,state,chamber,ensemble
0,2500,0.4837,0.0762,11.8678,11,0.0598,0.1075,0.0744,0.0435,0.0299,...,13,68,1.5858,1.5834,35,64,2,FL,congress,A0
1,5000,0.4837,0.0827,11.6842,11,0.0664,0.0973,0.0744,0.0500,0.0415,...,13,75,1.6487,1.6488,38,71,0,FL,congress,A0
2,7500,0.4837,0.0646,12.1917,12,0.0482,0.0673,0.0387,0.0319,0.0232,...,13,70,1.6020,1.6233,39,70,0,FL,congress,A0
3,10000,0.4837,0.0647,12.1873,12,0.0484,0.0745,0.0387,0.0320,0.0271,...,13,71,1.6085,1.5958,33,67,0,FL,congress,A0
4,12500,0.4837,0.0408,12.8575,13,0.0245,0.0399,0.0030,0.0081,0.0042,...,13,74,1.5676,1.5361,34,60,8,FL,congress,A0


This is the definition of anti-majoritarian in but not exported from `rdapy`.
No plans in any ensembles are anti-majoritarian by this definition.

In [5]:
AVG_SV_ERROR: float = 0.02

def is_antimajoritarian(Vf: float, Sf: float) -> bool:
    bDem = True if ((Vf < (0.5 - AVG_SV_ERROR)) and (Sf > 0.5)) else False
    bRep = True if (((1 - Vf) < (0.5 - AVG_SV_ERROR)) and ((1 - Sf) > 0.5)) else False

    return bDem or bRep

However, with this weak(er) definition, ...

In [9]:
def weak_is_antimajoritarian(Vf: float, Sf: float) -> bool:
    bDem = True if Vf < 0.5 and Sf > 0.5 else False
    bRep = True if (1 - Vf) < 0.5 and (1 - Sf) > 0.5 else False

    return bDem or bRep

In [11]:
anti_by_combo: Dict[str, int] = defaultdict(int)
plans_by_combo: Dict[str, int] = defaultdict(int)

ensembles = [e for e in ensembles if e not in ["A1", "A2", "A3", "A4", "Rev*"]]

for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx: str = row['state']
    chamber: str = row['chamber']
    combo: str = f"{xx}/{chamber}"

    plans_by_combo[combo] += 1

    ndistricts: int = DISTRICTS_BY_STATE[xx][chamber]
    Vf: float = row['estimated_vote_pct']
    Sf: float = row['estimated_seats'] / ndistricts    

    # if is_antimajoritarian(Vf, Sf):
    #     anti_by_combo[combo] += 1

    if weak_is_antimajoritarian(Vf, Sf):
        anti_by_combo[combo] += 1

for combo, count in anti_by_combo.items():
    total: int = plans_by_combo[combo]
    print(f"{combo}: {count:,} of {total:,} plans ({count/total:.1%}) are anti-majoritarian")
            



FL/upper: 12 of 219,999 plans (0.0%) are anti-majoritarian
MI/congress: 167,317 of 219,999 plans (76.1%) are anti-majoritarian
MI/upper: 205,681 of 219,999 plans (93.5%) are anti-majoritarian
MI/lower: 219,776 of 219,999 plans (99.9%) are anti-majoritarian
NC/congress: 7,931 of 219,999 plans (3.6%) are anti-majoritarian
NC/upper: 224 of 219,999 plans (0.1%) are anti-majoritarian
WI/congress: 210,977 of 219,999 plans (95.9%) are anti-majoritarian
WI/upper: 219,999 of 219,999 plans (100.0%) are anti-majoritarian
WI/lower: 219,999 of 219,999 plans (100.0%) are anti-majoritarian
